In [ ]:
import os
import uuid

from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

load_dotenv()

def upload_pdf_to_blob(pdf_bytes: bytes) -> str:
    container_name = os.getenv("AZURE_BLOB_CONTAINER_NAME")
    connection_string = os.getenv("AZURE_BLOB_CONNECTION_STRING")

    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=f"{str(uuid.uuid4())}.pdf")
    blob_client.upload_blob(pdf_bytes, overwrite=True)
    
    return blob_client.url

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

def protect_pdf(password: str, file_name: str):
    # 1. Load your existing PDF
    reader = PdfReader(f"temps/{file_name}.pdf")

    # 2. Copy pages into a new writer
    writer = PdfWriter()
    for page in reader.pages:
        writer.add_page(page)

    # 3. Encrypt with your chosen password
    writer.encrypt(user_password=password, owner_pwd=None, use_128bit=True)

    # 4. Write out the password-protected file. wb = "write binary" mode
    with open(f"temps/{file_name}-protected.pdf", "wb") as out_f:
        writer.write(out_f)

In [ ]:
import os
import requests

from docx2pdf import convert
from io import BytesIO

def convert_docx_to_pdf(remote_url: str, file_name: str):
    
    # Download DOCX to memory
    response = requests.get(remote_url)
    docx_bytes = BytesIO(response.content)
    
    # Ensure the temps directory exists
    os.makedirs("temps", exist_ok=True)
    
    # Save DOCX to a temporary file (docx2pdf requires a file path)
    with open(f"temps/{file_name}.docx", "wb") as temp_docx:
        temp_docx.write(docx_bytes.getbuffer())
        
    convert(f"temps/{file_name}.docx", f"temps/{file_name}.pdf")

In [ ]:
import uuid

remote_url = "https://emcdevstoragev2.blob.core.windows.net/public/26b49f14-0c66-48e5-ab10-d552ceef2256.docx"
file_name = str(uuid.uuid4())
password = "12345"

print(f"File name: {file_name}")

convert_docx_to_pdf(remote_url, file_name)
protect_pdf(password, file_name)

url = upload_pdf_to_blob(open(f"temps/{file_name}-protected.pdf", "rb").read())

print(f"Protected PDF URL: {url}")